In [2]:
import os
import numpy as np
from numpy import asarray
import PIL
from PIL import Image
import skimage
from skimage.transform import resize
from skimage.color import rgb2gray,gray2rgb
from skimage.filters import median,sato
from skimage.exposure import equalize_adapthist
from matplotlib import pyplot as plt
from skimage.io import imread

In [3]:
#CONNECT TO DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DIR="/content/drive/MyDrive/Colab Notebooks/images"
IMG_SIZE = 120

In [ ]:
CATEGORIES=["Afghan Hound","Beagle","Blenheim Spaniel","Japanese Spaniel","Maltese Terrier",
            "Papillon","Pom","Rhodesian Ridgeback","Samoyed","Shih-Tzu"]

In [ ]:
def preprocess(img):
  img = rgb2gray(img)
  img=median(img)
  img=sato(img)
  img = gray2rgb(img)
  return img

In [ ]:
#FIXING IMAGE SIZE TO 120x120
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        filename = os.path.join(path, img)
        img_array = asarray(Image.open(filename).convert('RGB'))
        img_array = resize(img_array, (IMG_SIZE, IMG_SIZE))
        plt.imsave(filename,img_array)

In [ ]:
#PREPROCESSING IMAGES
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        filename = os.path.join(path, img)
        img_array = asarray(Image.open(filename).convert('RGB'))
        augmented_array=preprocess(img_array)
        filename=filename[:-4] + '-augmented.jpg'
        plt.imsave(filename,augmented_array)

In [ ]:
#counts to total number of images
i=0
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
      i+=1

print("amount of images= "+str(i))

amount of images= 3137


In [ ]:
training_data=[]

In [ ]:
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        class_no = os.listdir(DIR).index(category)
        filename = os.path.join(path, img)
        img_array = asarray(skimage.io.imread(filename))
        training_data.append([img_array, class_no])


In [ ]:
X=[]
y=[]

for image,label in training_data:
    X.append(image)
    y.append(label)

In [ ]:
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)/255.0
y = np.array(y)


In [ ]:
X.shape


(3137, 120, 120, 3)

In [ ]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle= True)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.33, shuffle= True )

In [ ]:
base_model = InceptionV3(
                                weights='imagenet',
                                include_top=False,
                                input_shape=(IMG_SIZE, IMG_SIZE,3)
                                )

base_model.trainable=False

model = tf.keras.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2, 2, 2048)        21802784  
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                81930     
Total params: 21,884,714
Trainable params: 81,930
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
aug = ImageDataGenerator()
model.fit(aug.flow(X_train, y_train),validation_data = (X_val, y_val),epochs = 15,verbose=2)

Epoch 1/15
44/44 - 6s - loss: 3.9983 - accuracy: 0.3952 - val_loss: 3.1010 - val_accuracy: 0.4769
Epoch 2/15
44/44 - 2s - loss: 1.5189 - accuracy: 0.6318 - val_loss: 2.8948 - val_accuracy: 0.4942
Epoch 3/15
44/44 - 1s - loss: 0.8161 - accuracy: 0.7591 - val_loss: 2.6841 - val_accuracy: 0.5202
Epoch 4/15
44/44 - 2s - loss: 0.5258 - accuracy: 0.8365 - val_loss: 3.0373 - val_accuracy: 0.5202
Epoch 5/15
44/44 - 1s - loss: 0.3898 - accuracy: 0.8842 - val_loss: 2.9089 - val_accuracy: 0.5288
Epoch 6/15
44/44 - 1s - loss: 0.2720 - accuracy: 0.9204 - val_loss: 2.8427 - val_accuracy: 0.5418
Epoch 7/15
44/44 - 2s - loss: 0.2298 - accuracy: 0.9375 - val_loss: 3.2743 - val_accuracy: 0.5259
Epoch 8/15
44/44 - 2s - loss: 0.1746 - accuracy: 0.9538 - val_loss: 3.0540 - val_accuracy: 0.5317
Epoch 9/15
44/44 - 2s - loss: 0.0789 - accuracy: 0.9822 - val_loss: 2.8376 - val_accuracy: 0.5389
Epoch 10/15
44/44 - 2s - loss: 0.0541 - accuracy: 0.9950 - val_loss: 2.8452 - val_accuracy: 0.5389
Epoch 11/15
44/44 -

KeyboardInterrupt: ignored

In [ ]:
model.save("dog_classifier.model")

In [ ]:
the_model=tf.keras.models.load_model("dog_classifier.model")